In [2]:
# 异步计算
''' 
今天的计算机是⾼度并⾏的系统，由多个CPU核、多个GPU、多个处理单元组成。通常每个CPU核有多个线程，每
个设备通常有多个GPU，每个GPU有多个处理单元。总之，我们可以同时处理许多不同的事情，并且通常是在不
同的设备上。不幸的是，Python并不善于编写并⾏和异步代码，⾄少在没有额外帮助的情况下不是好选择。归根
结底，Python是单线程的，将来也是不太可能改变的。因此在诸多的深度学习框架中，MXNet和TensorFlow之
类则采⽤了⼀种异步编程（asynchronous programming）模型来提⾼性能，⽽PyTorch则使⽤了Python⾃
⼰的调度器来实现不同的性能权衡。对于PyTorch来说GPU操作在默认情况下是异步的。当你调⽤⼀个使
⽤GPU的函数时，操作会排队到特定的设备上，但不⼀定要等到以后才执⾏。这允许我们并⾏执⾏更多的计
算，包括在CPU或其他GPU上的操作。
因此，了解异步编程是如何⼯作的，通过主动地减少计算需求和相互依赖，有助于我们开发更⾼效的程序。
这使我们能够减少内存开销并提⾼处理器利⽤率

subprocess最早在2.4版本引入。
用来生成子进程，并可以通过管道连接他们的输入/输出/错误，以及获得他们的返回值。
'''
import os
import subprocess # subprocess 子进程
import numpy
import torch
from torch import nn
from d2l import torch as d2l

In [5]:
# 通过后端异步处理
''' 
作为热⾝，考虑⼀个简单问题：我们要⽣成⼀个随机矩阵并将其相乘。让我们在NumPy和PyTorch张量中都
这样做，看看它们的区别。请注意，PyTorch的tensor是在GPU上定义的

# for _ in range(n) 一般仅仅用于循环n次，不用设置变量，用 _ 指代临时变量，只在这个语句中使用一次
range(start, stop, step) 

#@save
class Benchmark: # 基准
    """⽤于测量运⾏时间"""
    def __init__(self, description='Done'):
        self.description = description # 描述,形容;种类,类型
    def __enter__(self):
        self.timer = d2l.Timer()
        return self
    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')
'''
# GPU计算热⾝
device = d2l.try_gpu() # 使用GPU
a = torch.randn(size=(1000, 1000), device=device) # 正态随机,默认均值为0，方差为1
b = torch.mm(a, a) # 矩阵相乘      torch.mul(a,b) 矩阵a,b对应位相乘
with d2l.Benchmark('numpy'): # 返回numpy矩阵相乘10次的时间
    for _ in range(10): # 循环10次
        a = numpy.random.normal(size=(1000, 1000)) # CPU上定义numpy
        b = numpy.dot(a, a)
with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device) # 在GPU上定义tensor
        b = torch.mm(a, a)

numpy: 0.7067 sec
torch: 0.0012 sec


In [6]:
''' 
通过PyTorch的基准输出⽐较快了⼏个数量级。NumPy点积是在CPU上执⾏的，⽽PyTorch矩阵乘法是
在GPU上执⾏的，后者的速度要快得多。但巨⼤的时间差距表明⼀定还有其他原因。默认情况下，GPU操
作在PyTorch中是异步的。强制PyTorch在返回之前完成所有计算，这种强制说明了之前发⽣的情况：计算是
由后端执⾏，⽽前端将控制权返回给了Python
---------------------------------------------------------------------------------------

torch.cuda.synchronize()
start = time.time()
result = model(input)
torch.cuda.synchronize()
end = time.time()

之前发现算法处理完的tensor从gpu转到cpu很耗时，一直找不到解决方法，后面看到有人说测试时间要先同步
才发现耗时的不是这个转换过程,这是因为CUDA kernel函数是异步的，所以不能直接在CUDA函数两端加上
time.time()测试时间，这样测出来的只是调用CUDA api的时间，不包括GPU端运行的时间。
我们需要要加上线程同步函数，等待kernel中所有线程全部执行完毕再执行CPU端后续指令。
上面代码我们将同步指令加在了python端，用的是torch.cuda.synchronize函数。

即pytorch异步执行,调用函数后操作会排队到指定的设备上,不会等方法执行完成,而是直接调用执行后续代码
方法执行完成后主要通过状态通知主线程或回调处理这次异步方法执行的结果
'''
'这才是等待程序执行完成后所用的时间,上述极短的时间为调用duda api的时间'
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device) # 等待当前设备上所有流中的所有核心完成 synchronize 线程同步函数


Done: 0.2081 sec


In [7]:
''' 
⼴义上说，PyTorch有⼀个⽤于与⽤⼾直接交互的前端（例如通过Python），还有⼀个由系统⽤来执⾏计算的
后端。如 图12.2.1所⽰，⽤⼾可以⽤各种前端语⾔编写PyTorch程序，如Python和C++。不管使⽤的前端编程
语⾔是什么，PyTorch程序的执⾏主要发⽣在C++实现的后端。由前端语⾔发出的操作被传递到后端执⾏。后
端管理⾃⼰的线程，这些线程不断收集和执⾏排队的任务。请注意，要使其⼯作，后端必须能够跟踪计算图
中各个步骤之间的依赖关系。因此，不可能并⾏化相互依赖的操作。
'''
# 让我们看另⼀个简单例⼦，以便更好地理解依赖关系图
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='cuda:0')

In [ ]:
''' 
上⾯的代码⽚段在 图12.2.2中进⾏了说明。每当Python前端线程执⾏前三条语句中的⼀条语句时，它只是将
任务返回到后端队列。当最后⼀个语句的结果需要被打印出来时，Python前端线程将等待C++后端线程完成
变量z的结果计算。这种设计的⼀个好处是Python前端线程不需要执⾏实际的计算。因此，不管Python的性
能如何，对程序的整体性能⼏乎没有影响。图12.2.3演⽰了前端和后端如何交互

障碍器与阻塞器
改进计算
-----------------------------------------------------------------------------------------
⼩结
• 深度学习框架可以将Python前端的控制与后端的执⾏解耦，使得命令可以快速地异步插⼊后端、并⾏
执⾏。
• 异步产⽣了⼀个相当灵活的前端，但请注意：过度填充任务队列可能会导致内存消耗过多。建议对每个
⼩批量进⾏同步，以保持前端和后端⼤致同步。
• 芯⽚供应商提供了复杂的性能分析⼯具，以获得对深度学习效率更精确的洞察
'''